<a href="https://colab.research.google.com/github/agroimpacts/VegMapper/blob/devel-calval/prepare_train_val_ref_sample_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a single training/validation/test set from multiple Collect Earth projects

 

### Table of Contents

* [Overview](#overview)
* [Set-up](#setup)
* [Sample preparation](#sample-prep)
    * [Read-in, reshape, and recode](#reshape-recode) 
    * [Simplify the classes](#simplify)
    * [Calculate sample agreement](#agreement)
* [Split the dataset](#split)
    * [Combine and convert to spatial](#combine)
* [Key to variable names](#key)


## Overview <a class="anchor" id="overview"></a>
This notebook demonstrates how several Collect Earth Online projects can be:

1. Recoded and reshaped into a structure more tractable for modeling; 
2. Merged into a single dataset that provide a single label for each sample point and an estimate of label uncertainty;
3. Split into training, validation, and test (or map reference) samples. 

The data used in this demonstration are the results of three Collect Earth Online projects that were captured over the Department of Ucayali, Peru. Each project represents the efforts of an individual (or group of individuals working in the same project) to label 1350 points, classifying each into 1 of 4 classes: not oil palm; young oil palm; mature oil palm;  unsure. The datasets preserve all the information from these projects, although user email addresses were anonymized.

## Set-up <a class="anchor" id="setup"></a>

This notebook uses an `R` kernel, which was set up in a `conda` environment using the following commands to set up and activate the environment:

```bash
(base) % conda create -n r_env r-essentials r-base
(base) % conda activate r_env
```

The necessary `R` packages were then installed as follows:
```bash
(r_env) % conda install -c conda-forge r-sf
(r_env) % install -c r r-catools
(r_env) % install -c conda-forge r-raster
(r_env) % install -c conda-forge r-geojsonio
```

## Sample preparation <a class="anchor" id="sample-prep"></a>
Load packages with messages suppressed.

In [ ]:
import_library = function(lib_name){
    suppressWarnings(suppressMessages(require(lib_name, character.only = TRUE)))
}

import_library('dplyr')
import_library('sf')
import_library('caTools')
import_library('ggplot2')
import_library('raster')


print('break')
install.packages('raster')

### Read-in, reshape, and recode classes <a class="anchor" id="reshape-recode"></a>
The first step was to combine the three datasets into a single dataset, with the columns from each of the three CEO projects, and to recode the four classes into a single column with values 0 (not oil palm),  1 (young oil palm), 2 (mature oil palm), 3 (unsure). At this step, we end up with 3 columns, 1 per completed CEO project: `cl1` = samples from project 1, `cl2` = samples from project 2, `cl3` = samples from project 3. Each column contains the recoded classes (note the renaming of the columns is done in the next code chunk). 

In [ ]:
## need to import these survey files first (to Files on left)
fs <- c("/ceo-survey-user1.csv",
        "/ceo-survey-user2.csv",
        "/ceo-survey-user3.csv")
print("fs")
print(fs) 


# read datasets into list, keep the most important columns, and do some renaming
data_list <- lapply(fs, function(x) {
  dat <- readr::read_csv(x, col_types = readr::cols()) %>% 
    select(pl_plotid, pl_cluster, email, center_lon, center_lat, 
           contains("Oil")) %>% 
    rename(x = center_lon, y = center_lat, clust = pl_cluster) %>% 
    rename_at(vars(contains("Oil")), list(~gsub("Oil Palm|\\?\\:| ","", .))) %>% 
    mutate(class = case_when(
      Young == 100 ~ 1, 
      Mature == 100 ~ 2,
      Not == 100 ~ 0, 
      NotSure == 100 ~ 3
    )) %>% 
    rename(id = pl_plotid) %>% 
    select(id, clust, class)
})

# combine into single dataset, conttains just the plot id, the stratum (clust), 
# and the recoded classification columns from each input
sample_combined <- purrr::reduce(data_list, left_join, by = c("id", "clust"))
nms <- names(sample_combined)
nms[grep("class", nms)] <- paste0("cl", 1:length(grep("class", nms)))
names(sample_combined) <- nms

head(sample_combined)

### Simplify the classes <a class="anchor" id="simplify"></a>

In this step, a single classification is created by finding the modal class for each sample point across the 3 groups' results. This creates a new `class` column, which provides the class from the majority opinion. 

We repeat this same step again after first collapsing, within each of the `cl1:cl3` columns, the two oil palm classes into a single *oil palm* class with value = 1--*not oil palm* remains 0, and *unsure* remains 3. The modal function was re-run to create a new consensus class, called `class2`. We recommend that `class2` be used for modelling, while `class` may be useful for understanding error patterns. 

In [ ]:
# reshape and assign class as most frequently occurring value

# need to check nas for 'modal'

modal <- function(x) {
  ux <- unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}



op_reclass <- function(x) {
  modal(ifelse(x %in% 1:2, 1, x))
}
samples_red <- sample_combined %>% 
  rowwise() %>% 
  mutate(class = modal(c_across(cl1:cl3))) %>% 
  mutate(class2 = op_reclass(c_across(cl1:cl3))) %>% 
  select(id, clust, class, class2, cl1, cl2, cl3) %>% 
  ungroup()

write.csv(samples_red, 'test_samples_red.csv')


### Calculate sample agreement <a class="anchor" id="agreement"></a>
The next step was to calculate some agreement metrics across the three groups' samples. The primary approach was to calculate the proportion of labelling teams that selected the modal class. Since there were just three teams in this example, values were either 0.333, 0.667, 1. This agreement was calculated across for both the original classification scheme (class: 0-3) and the simplifed scheme (), with columns `agree` and `agree2` providing the respective proportions for each observation. 

In [35]:
# function to calculate agreement
agree_func <- function(x, y) {
  y <- y[!is.na(y)]
  length(which(y == x)) / length(y)
}

# agreement within original classification scheme
agreement <- sapply(1:nrow(samples_red), function(x) {  # x <- 1
  dat <- samples_red %>% slice(x) %>% select(class:cl3) %>% as.data.frame()
  agree_func(dat$class, dat[c("cl1", "cl2", "cl3")])
})

# agreement in reduced classification scheme
agreement2 <- sapply(1:nrow(samples_red), function(x) {  # x <- 2
  dat <- samples_red %>% slice(x) %>% 
    select(class:cl3) %>% 
    mutate_at(vars(cl1:cl3), ~ifelse(. %in% 1:2, 1, .)) %>% 
    as.data.frame()
  agree_func(dat$class2, dat[c("cl1", "cl2", "cl3")])
})

# combine
samples_redf <- samples_red %>% 
  mutate(agree = agreement, agree2 = agreement2) %>% 
  select(id, clust, class, agree, class2, agree2, !!names(.))

head(samples_redf)
write.csv(samples_redf, 'test_samples_redf.csv')


id,clust,class,agree,class2,agree2,cl1,cl2,cl3
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2,1.0000000,1,1,2,2,NA
2,1,0,1.0000000,0,1,0,0,0
3,0,0,1.0000000,0,1,0,0,0
4,1,2,0.6666667,1,1,2,2,1
5,0,0,1.0000000,0,1,0,0,0
6,0,0,1.0000000,0,1,0,0,0


We can then calculate the average agreement per sample to get a sense of the uncertainty in labels for each class, for all 4 classes 

In [ ]:
sample_stats <- samples_redf %>% group_by(class) %>% 
  summarize(count = n(), agree = mean(agree))
sample_stats

And for the reduced set of classes

In [ ]:
sample_stats2 <- samples_redf %>% group_by(class2) %>% 
  summarize(count = n(), agree = mean(agree2))
sample_stats2

## Split the dataset <a class="anchor" id="split"></a>

Here we split the dataset into three parts for model training (60% of the sample), validation (20%), and final assessment (the 20% set aside as the test or map reference dataset).

The splits are confined to the usable sample, which is defined as samples not falling into class 3 and those with at least 2/3 observers agreeing on the class. This decision is made based on the simplified sample scheme (class2), rather than the full scheme (class). The resulting splits are denoted in a column called `usage` (this is distinct from the column `use`, which was used to filter out unusable observations). 

Values of "unusable" in the `usage` column indicate observations that were not usable because of their low agreement or uncertain class.  They are included here for completeness, and in case they help with evaluation


In [ ]:
full_sample <- samples_redf %>% 
  mutate(use = ifelse(agree2 > 0.5 & class2 != 3, 1, 0)) %>% 
  # mutate(ref_use = ifelse(clust == 11, 0, 1)) %>% 
  select(id, clust, class, agree, class2, agree2, use,
         !!names(.)) %>% 
  mutate(usage = NA) %>% 
  select(id, clust, use, usage, class2, agree2, class, agree, 
         cl1:cl3)

# do a join to get the coordinates for each site
# read back in one of the datasets to get sample id and coordinates 
xydat <- readr::read_csv(fs[1], col_types = readr::cols()) %>% 
  select(pl_plotid, center_lon, center_lat) %>% 
  rename(id = pl_plotid, x = center_lon, y = center_lat)  # rename 
full_sample <- left_join(full_sample, xydat, by = "id")  # join

# filtered down to usable
filtered_sample <- full_sample %>% 
  filter(use == 1)

# split off train from rest - we included the points drawn from 
# previously mapped locations (cluster 11) in the eligible
# pool of validation/reference points here
set.seed(10)
samp <- sample.split(filtered_sample$class2, SplitRatio = 0.6)
train <- subset(filtered_sample, samp == TRUE) %>% 
  mutate(usage = "train")  # train split
valref <- subset(filtered_sample, samp == FALSE)  # val/ref split

# split into validation and test
set.seed(10)
samp <- sample.split(valref$class2, SplitRatio = 0.5)  
val <- subset(valref, samp == TRUE) %>% mutate(usage = "validate")
ref <- subset(valref, samp == FALSE) %>% mutate(usage = "map reference/test")

# Check for duplicate in each
any(val$id %in% ref$id)
any(val$id %in% train$id)
any(ref$id %in% train$id)

### Combine and convert to spatial <a class="anchor" id="combine"></a>

The ineligible portion of the sample is also added back for completeness

In [ ]:
trainvalref <- bind_rows(train, val, ref)
print(trainvalref[,c('id', 'clust', 'agree', 'x', 'y')])
full_samplef <- full_sample %>% filter(!id %in% !!(trainvalref$id)) %>% 
  mutate(usage = "unusable") %>% 
  bind_rows(trainvalref, .) # %>% st_as_sf(coords = c("x", "y"))

print(full_samplef)

Write it out as needed (after uncommenting)

In [ ]:
# st_write(
#   full_samplef, 
#   dsn = "data/ucayali_sample_fin.geojson", 
#   delete_dsn = TRUE, quiet = TRUE
# )

Here is the count distribution by class and usage

In [ ]:
as_tibble(full_samplef) %>% group_by(usage, class2) %>% count()

And their locations on a map

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
ggplot(full_samplef) + 
  geom_point(aes(x = x, y = y, color = usage), size = 1) + 
  theme_linedraw()

## Key to variable names <a class="anchor" id="key"></a>

The variable names and their descriptions are as follows:

- id = Sample ID (corrected after randomized ordering)
- clust = Stratification cluster (0, 1, 11 [previously digitized areas])
- use = Usability of sample: 1 = suitable for training/validation/map reference, 0 = unusable
- usage = Categorized into training, validation, and map reference (test) splits
- class2: Classification by consensus on simplified oil palm class
- agree2: Agreement per observation, assessed using simplified classification scheme
- class: Consensus classification using all four classes (not recommended for modelling)
- agree: Agreement per observation, assessed using full classification scheme
- cl1: Labeller 1's sample
- cl2: Labeller 2 sample
- cl3: Labeller 3 sample
- geometry: X and Y coordinates as simple features